In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
! unzip project.zip

Archive:  project.zip
  inflating: LICENSE                 
 extracting: scores                  
  inflating: test_bodies.csv         
   creating: __MACOSX/
  inflating: __MACOSX/._test_bodies.csv  
   creating: utils/
  inflating: utils/generate_test_splits.py  
   creating: __MACOSX/utils/
  inflating: __MACOSX/utils/._generate_test_splits.py  
  inflating: utils/system.py         
  inflating: __MACOSX/utils/._system.py  
 extracting: utils/__init__.py       
   creating: utils/__pycache__/
  inflating: utils/__pycache__/dataset.cpython-36.pyc  
  inflating: utils/__pycache__/generate_test_splits.cpython-36.pyc  
  inflating: utils/__pycache__/score.cpython-36.pyc  
  inflating: utils/__pycache__/system.cpython-36.pyc  
  inflating: utils/__pycache__/__init__.cpython-36.pyc  
  inflating: utils/dataset.py        
  inflating: __MACOSX/utils/._dataset.py  
  inflating: utils/score.py          
  inflating: __MACOSX/utils/._score.py  
  inflating: competition_test_stances.csv  
  in

In [1]:
import sys
import os
import numpy as np
from numpy import array
from numpy import asarray
from tensorflow.keras.utils import to_categorical
import pandas as pd
import tensorflow as tf
#from gensim.models import Word2Vec
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional, Flatten
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import callbacks, regularizers
from utils.dataset import DataSet
from utils.generate_test_splits import train_vali_split, get_stances_for_folds
from utils.score import report_score, LABELS, score_submission
from tensorflow.keras.utils import plot_model
from utils.system import parse_params, check_version
import re
import string

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
LSTM_DIM = 300
EMBEDDING_DIM = 300

In [3]:

#Load the training dataset and generate folds
d = DataSet()
training, hold_out = train_vali_split(d)
training_stances, hold_out_stances = get_stances_for_folds(d,training,hold_out)
# Load the competition dataset
competition_dataset = DataSet("competition_test")

Reading dataset
Total stances: 49972
Total bodies: 1683
Reading dataset
Total stances: 25413
Total bodies: 904


In [4]:
def prepareData(dataset, stances):
    headline = []
    body = []
    stanceFinal = []
    for stance in stances:
        headline.append(stance['Headline'])
        body.append(dataset.articles[stance['Body ID']])
        stanceFinal.append(stance['Stance'])
    return headline, body, stanceFinal
        

In [5]:
train_headline, train_body, train_stanceFinal = prepareData(d, training_stances)
hold_out_headline, hold_out_body, hold_out_stanceFinal = prepareData(d, hold_out_stances)

In [6]:
competition_headline, competition_body, competition_stanceFinal = prepareData(competition_dataset, competition_dataset.stances)

In [7]:
def stance_to_onehot(stance):
    li = []
    LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
    for i in stance:
        if i == LABELS[0]:
            li.append([1,0,0,0])
        elif i == LABELS[1]:
            li.append([0,1,0,0])
        elif i == LABELS[2]:
            li.append([0,0,1,0])
        elif i == LABELS[3]:
            li.append([0,0,0,1])
    return np.array(li)

In [8]:
competition_stance_onehot = stance_to_onehot(competition_stanceFinal)
train_stance_onehot = stance_to_onehot(train_stanceFinal)
hold_out_stance_onehot = stance_to_onehot(hold_out_stanceFinal)

In [9]:
token_pattern = r"(?u)\b\w\w+\b"
def get_tokens(line,
                    token_pattern=token_pattern):
    token_pattern = re.compile(token_pattern, flags = re.UNICODE)
    tokens = [x.lower() for x in token_pattern.findall(line)]
    
    return tokens

In [10]:
train_headline = [get_tokens(text) for text in train_headline]
hold_out_headline = [get_tokens(text) for text in hold_out_headline]
train_body = [get_tokens(text) for text in train_body]
hold_out_body = [get_tokens(text) for text in hold_out_body]
competition_headline = [get_tokens(text) for text in competition_headline]
competition_body = [get_tokens(text) for text in competition_body]

In [11]:
tokenizer = Tokenizer()
corpus = train_headline+train_body+hold_out_headline+hold_out_body+competition_headline+competition_body
tokenizer.fit_on_texts([' '.join(seq) for seq in corpus])
vocab_size = len(tokenizer.word_index) + 1

In [12]:
vocab_size

29419

In [13]:
train_headline = tokenizer.texts_to_sequences(train_headline)
train_body = tokenizer.texts_to_sequences(train_body)
hold_out_headline = tokenizer.texts_to_sequences(hold_out_headline)
hold_out_body = tokenizer.texts_to_sequences(hold_out_body)
competition_headline = tokenizer.texts_to_sequences(competition_headline)
competition_body = tokenizer.texts_to_sequences(competition_body)

In [14]:
MAX_BODY_LENGTH = int(np.percentile([len(doc) for doc in train_body+hold_out_body+competition_body],10))

print('10th Percentile body Length:', MAX_BODY_LENGTH)

10th Percentile body Length: 101


In [15]:
MAX_HEADLINE_LENGTH = int(np.percentile([len(doc) for doc in train_headline+hold_out_headline], 90))

print('90th Percentile headline Length:', MAX_HEADLINE_LENGTH)
MAX_HEADLINE_LENGTH =15
MAX_BODY_LENGTH = int(np.mean([len(doc) for doc in train_body+hold_out_body]))

print('90th Percentile body Length:', MAX_BODY_LENGTH)
MAX_BODY_TRUNC = 50

90th Percentile headline Length: 16
90th Percentile body Length: 361


In [16]:
train_headline = pad_sequences(train_headline, maxlen=MAX_HEADLINE_LENGTH, padding='post', truncating='post')
hold_out_headline = pad_sequences(hold_out_headline, maxlen=MAX_HEADLINE_LENGTH, padding='post', truncating='post')
train_first_body = pad_sequences(train_body, maxlen=MAX_BODY_TRUNC, padding='post', truncating='post')
train_last_body = pad_sequences(train_body, maxlen=MAX_BODY_TRUNC, padding='post', truncating='pre')
hold_out_first_body = pad_sequences(hold_out_body, maxlen=MAX_BODY_TRUNC, padding='post', truncating='post')
hold_out_last_body = pad_sequences(hold_out_body, maxlen=MAX_BODY_TRUNC, padding='post', truncating='pre')
competition_headline = pad_sequences(competition_headline, maxlen=MAX_HEADLINE_LENGTH, padding='post', truncating='post')
competition_first_body = pad_sequences(competition_body, maxlen=MAX_BODY_TRUNC, padding='post', truncating='post')
competition_last_body = pad_sequences(competition_body, maxlen=MAX_BODY_TRUNC, padding='post', truncating='pre')



In [17]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.42B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
del embeddings_index

Loaded 1917494 word vectors.


In [18]:
def builtWeightSharingModel(drop_out_rate=0.5,dense_neuron = 32):
    headline_input = Input(shape=(MAX_HEADLINE_LENGTH,), name='headline_input')
    first_body_input = Input(shape=(MAX_BODY_TRUNC,), name='first_body_input')
    last_body_input = Input(shape=(MAX_BODY_TRUNC,), name='last_body_input')
    e = Embedding(input_dim=len(tokenizer.word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights = [embedding_matrix], trainable=False, name='headline_word_embedding_layer', 
                          mask_zero=True)
    headline_embedding = e(headline_input)
    first_body_embedding = e(first_body_input)
    last_body_embedding = e(last_body_input)
    encoder_outputs, state_h, state_c = LSTM(LSTM_DIM, return_sequences=False, return_state = True,dropout=0.5,recurrent_dropout = 0.5, name='lstm_layer_encoder')(headline_embedding)
    encoder_states = [state_h, state_c]
    decoder = LSTM(LSTM_DIM, return_sequences=False, return_state = True,dropout=0.5,recurrent_dropout = 0.5, name='lstm_layer_first_body')
    first_body_outputs, _, _ = decoder(first_body_embedding, initial_state=encoder_states)
    last_body_outputs, _, _ = decoder(last_body_embedding, initial_state=encoder_states)
    x = tf.keras.layers.concatenate([first_body_outputs, last_body_outputs])
    x = Dropout(rate=drop_out_rate, name='dropout_0')(x)
    #x = Dense(dense_neuron, activation='relu')(x)
    #x = Dropout(rate=drop_out_rate, name='dropout_1')(x)
    #x = Dense(dense_neuron, activation='relu')(x)
    #x = Dropout(rate=drop_out_rate, name='dropout_2')(x)

    # And finally we add the main logistic regression layer
    main_output = Dense(4, activation='softmax', name='main_output')(x)
    model = Model(inputs=[headline_input, first_body_input, last_body_input], outputs=main_output)
    model.summary()
    return model

In [0]:
def builtConditionalModel(drop_out_rate=0.5,dense_neuron = 32):
    headline_input = Input(shape=(MAX_HEADLINE_LENGTH,), name='headline_input')
    first_body_input = Input(shape=(MAX_BODY_TRUNC,), name='first_body_input')
    #last_body_input = Input(shape=(MAX_BODY_TRUNC,), name='last_body_input')
    e = Embedding(input_dim=len(tokenizer.word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights = [embedding_matrix], trainable=False, name='headline_word_embedding_layer', 
                          mask_zero=True)
    headline_embedding = e(headline_input)
    first_body_embedding = e(first_body_input)
    #last_body_embedding = e(last_body_input)
    encoder_outputs, state_h, state_c = LSTM(LSTM_DIM, return_sequences=False, return_state = True,dropout=0.5,recurrent_dropout = 0.5, name='lstm_layer_encoder')(headline_embedding)
    encoder_states = [state_h, state_c]
    decoder = LSTM(LSTM_DIM, return_sequences=False, return_state = True,dropout=0.5,recurrent_dropout = 0.5, name='lstm_layer_first_body')
    first_body_outputs, _, _ = decoder(first_body_embedding, initial_state=encoder_states)
    #last_body_outputs, _, _ = decoder(last_body_embedding, initial_state=encoder_states)
    #x = tf.keras.layers.concatenate([first_body_outputs, last_body_outputs])
    x = Dropout(rate=drop_out_rate, name='dropout_0')(first_body_outputs)
    #x = Dense(dense_neuron, activation='relu')(x)
    #x = Dropout(rate=drop_out_rate, name='dropout_1')(x)
    #x = Dense(dense_neuron, activation='relu')(x)
    #x = Dropout(rate=drop_out_rate, name='dropout_2')(x)

    # And finally we add the main logistic regression layer
    main_output = Dense(4, activation='softmax', name='main_output')(x)
    model = Model(inputs=[headline_input, first_body_input], outputs=main_output)
    model.summary()
    return model

In [0]:
def builtCompositeModel(drop_out_rate=0.5,dense_neuron = 32):
    headline_input = Input(shape=(MAX_HEADLINE_LENGTH,), name='headline_input')
    first_body_input = Input(shape=(MAX_BODY_TRUNC,), name='first_body_input')
    last_body_input = Input(shape=(MAX_BODY_TRUNC,), name='last_body_input')
    headline_embedding = Embedding(input_dim=len(tokenizer.word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights = [embedding_matrix], trainable=False, name='headline_word_embedding_layer', 
                          mask_zero=True)(headline_input)
    first_body_embedding = Embedding(input_dim=len(tokenizer.word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights = [embedding_matrix], trainable=False, name='first_body_word_embedding_layer', 
                          mask_zero=True)(first_body_input)
    last_body_embedding = Embedding(input_dim=len(tokenizer.word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights = [embedding_matrix], trainable=False, name='last_bodyword_embedding_layer', 
                          mask_zero=True)(last_body_input)
    encoder_outputs, state_h, state_c = LSTM(LSTM_DIM, return_sequences=False, return_state = True,dropout=0.5,recurrent_dropout = 0.5, name='lstm_layer_encoder')(headline_embedding)
    encoder_states = [state_h, state_c]
    first_body_outputs, _, _ = LSTM(LSTM_DIM, return_sequences=False, return_state = True,dropout=0.5,recurrent_dropout = 0.5, name='lstm_layer_first_body')(first_body_embedding, initial_state=encoder_states)
    last_body_outputs, _, _ = LSTM(LSTM_DIM, return_sequences=False, return_state = True,dropout=0.5,recurrent_dropout = 0.5, name='lstm_layer_last_body')(last_body_embedding, initial_state=encoder_states)
    x = tf.keras.layers.concatenate([first_body_outputs, last_body_outputs])
    x = Dropout(rate=drop_out_rate, name='dropout_1')(x)
    #x = Dense(dense_neuron, activation='relu')(x)
    #x = Dropout(rate=drop_out_rate, name='dropout_1')(x)
    #x = Dense(dense_neuron, activation='relu')(x)
    #x = Dropout(rate=drop_out_rate, name='dropout_2')(x)

    # And finally we add the main logistic regression layer
    main_output = Dense(4, activation='softmax', name='main_output')(x)
    model = Model(inputs=[headline_input, first_body_input, last_body_input], outputs=main_output)
    model.summary()
    return model

In [ ]:
def trainModel(model,train_input,vali_input, batch=128,ep=50):
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    output_directory = ''
    model_checkpoint = callbacks.ModelCheckpoint(os.path.join(output_directory , 'weights.{epoch:02d}-{val_loss:.2f}.hdf5'),save_best_only=True,mode='auto',period=10)
    '''
    model.fit([np.concatenate((train_headline, hold_out_headline), axis=0), np.concatenate((train_first_body, hold_out_first_body), axis=0), np.concatenate((train_last_body, hold_out_last_body), axis=0)], np.concatenate((train_stance_onehot, hold_out_stance_onehot), axis=0),
          batch_size=batch,
          epochs=ep,
          validation_data=([competition_headline, competition_first_body, competition_last_body], competition_stance_onehot),callbacks=[model_checkpoint])
    '''
    model.fit(train_input, train_stance_onehot,
          batch_size=batch,
          epochs=ep,
          validation_data=(vali_input, hold_out_stance_onehot),callbacks=[model_checkpoint])
    #'''
    return model

In [19]:
model = trainModel(builtWeightSharingModel(drop_out_rate=0.5),[train_headline, train_first_body, train_last_body],[hold_out_headline, hold_out_first_body, hold_out_last_body])

NameError: name 'trainModel' is not defined

In [0]:
model = trainModel(model,[train_headline, train_first_body, train_last_body],[hold_out_headline, hold_out_first_body, hold_out_last_body])

W0725 17:20:33.336267 139669666088832 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Train on 40350 samples, validate on 9622 samples
Epoch 1/50
40350/40350 [==============================] - 87s 2ms/sample - loss: 0.2969 - acc: 0.8879 - val_loss: 0.5554 - val_acc: 0.7986
Epoch 2/50
40350/40350 [==============================] - 85s 2ms/sample - loss: 0.2971 - acc: 0.8865 - val_loss: 0.5357 - val_acc: 0.8021
Epoch 3/50
40350/40350 [==============================] - 85s 2ms/sample - loss: 0.2958 - acc: 0.8888 - val_loss: 0.5294 - val_acc: 0.8060
Epoch 4/50
40350/40350 [==============================] - 85s 2ms/sample - loss: 0.2895 - acc: 0.8910 - val_loss: 0.5047 - val_acc: 0.8138
Epoch 5/50
40350/40350 [==============================] - 85s 2ms/sample - loss: 0.2912 - acc: 0.8911 - val_loss: 0.5147 - val_acc: 0.8084
Epoch 6/50
40350/40350 [==============================] - 85s 2ms/sample - loss: 0.2855 - acc: 0.8929 - val_loss: 0.5326 - val_acc: 0.8034
Epoch 7/50
40350/40350 [==============================] - 85s 2ms/sample - loss: 0.2840 - acc: 0.8932 - val_loss: 0.5

In [0]:
def y_to_stance(onehot):
    LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
    return [LABELS[index] for index in onehot]

def evaluate(predicted, actual):
    
    predicted = y_to_stance(predicted)
    print("Scores on the test set")
    report_score(actual,predicted)

In [0]:
model.save('weight_sharing')

In [30]:
print('weight sharing')
pred = model.predict([hold_out_headline, hold_out_first_body,hold_out_last_body])
y_classes = pred.argmax(axis=-1)
actual = hold_out_stanceFinal
evaluate(y_classes, actual)

pred = model.predict([competition_headline, competition_first_body,competition_last_body])
y_classes = pred.argmax(axis=-1)
actual = competition_stanceFinal
evaluate(y_classes, actual)


weight sharing
Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    386    |     2     |    239    |    135    |
-------------------------------------------------------------
| disagree  |    86     |     1     |    60     |    15     |
-------------------------------------------------------------
|  discuss  |    145    |     8     |   1382    |    265    |
-------------------------------------------------------------
| unrelated |    466    |     0     |    437    |   5995    |
-------------------------------------------------------------
Score: 3402.75 out of 4448.5	(76.49207598066764%)
Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    813 

In [0]:
model = trainModel(builtCompositeModel(drop_out_rate=0.5),[train_headline, train_first_body, train_last_body],[hold_out_headline, hold_out_first_body, hold_out_last_body])

W0723 23:09:45.756192 140532354664320 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
headline_input (InputLayer)     [(None, 15)]         0                                            
__________________________________________________________________________________________________
first_body_input (InputLayer)   [(None, 50)]         0                                            
__________________________________________________________________________________________________
headline_word_embedding_layer ( (None, 15, 300)      8825700     headline_input[0][0]             
__________________________________________________________________________________________________
last_body_input (InputLayer)    [(None, 50)]         0                                            
____________________________________________________________________________________________

In [0]:
print('COMPOSITE')
pred = model.predict([hold_out_headline, hold_out_first_body,hold_out_last_body])
y_classes = pred.argmax(axis=-1)
actual = hold_out_stanceFinal
evaluate(y_classes, actual)

pred = model.predict([competition_headline, competition_first_body,competition_last_body])
y_classes = pred.argmax(axis=-1)
actual = competition_stanceFinal
evaluate(y_classes, actual)

COMPOSITE


NameError: ignored

In [0]:
model = trainModel(builtConditionalModel(drop_out_rate=0.5),[train_headline, train_first_body],[hold_out_headline, hold_out_first_body])

W0724 00:56:45.422139 140532354664320 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_body_input (InputLayer)   [(None, 50)]         0                                            
__________________________________________________________________________________________________
headline_input (InputLayer)     [(None, 15)]         0                                            
__________________________________________________________________________________________________
headline_word_embedding_layer ( multiple             8825700     headline_input[0][0]             
                                                                 first_body_input[0][0]           
__________________________________________________________________________________________________
lstm_layer_encoder (LSTM)       [(None, 300), (None, 721200      headline_word_embedding_lay

In [0]:
print('Conditional')
pred = model.predict([hold_out_headline, hold_out_first_body])
y_classes = pred.argmax(axis=-1)
actual = hold_out_stanceFinal
evaluate(y_classes, actual)

pred = model.predict([competition_headline, competition_first_body])
y_classes = pred.argmax(axis=-1)
actual = competition_stanceFinal
evaluate(y_classes, actual)

Conditional
Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    555    |    11     |    129    |    67     |
-------------------------------------------------------------
| disagree  |    62     |    70     |    23     |     7     |
-------------------------------------------------------------
|  discuss  |    235    |    70     |   1394    |    101    |
-------------------------------------------------------------
| unrelated |    170    |    18     |    161    |   6549    |
-------------------------------------------------------------
Score: 3788.75 out of 4448.5	(85.16915814319434%)
Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    904    

In [0]:
h= []
b = []
pred = model.predict([competition_headline, competition_first_body, competition_last_body])
y_classes = pred.argmax(axis=-1)
predicted = y_to_stance(y_classes)
for stance in competition_dataset.stances:
    h.append(stance['Headline'])
    b.append(stance['Body ID'])
np.savetxt("headline.csv", h, delimiter=",",fmt='%s')
np.savetxt("body_id.csv", b, delimiter=",",fmt='%s')
np.savetxt("stance.csv", predicted, delimiter=",",fmt='%s')

In [0]:
pd.DataFrame({'Headline':h,'Body ID':b,'Stance':predicted }).to_csv('/content/drive/My Drive/try/weight_sharing.csv',index=False)